### Melbourne Open Data
<div class="Melbourne Open Data - Business Establishments">
  <i>Dataset:</i> Business establishments location and industry classification 374210 records
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/business-establishments-with-address-and-industry-classification/information/" target="_blank">Dataset Link</a>
</div>

##### Open Data is provided under a creative commons licence
- Free to share, adapt data for any purpose as long as credit is given to Melbourne City Council

In [ ]:
#Libraries
!pip -q install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 6.0 MB/s eta 0:00:00


In [ ]:
from io import StringIO
import requests

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load libraries
import os
import warnings
warnings.filterwarnings('ignore')
import requests
from urllib.request import urlopen
import json
from tqdm import tqdm

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install tqdm # Progress bar
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Download datasets
def download_dataset(dataset_id, base_url='https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'):
    format = 'csv'
    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC'
    }

    with requests.get(url, params=params, stream=True) as response:
        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            chunk_size = 1024  # 1KB per chunk
            progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True, desc=f"Downloading {dataset_id}")

            content = bytearray()
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:  # filter out keep-alive new chunks
                    content.extend(chunk)
                    progress_bar.update(len(chunk))

            progress_bar.close()
            data = pd.read_csv(StringIO(content.decode('utf-8')), delimiter=';')
            return data
        else:
            print(f'Request failed with status code {response.status_code}')
            return None

# Dataset IDs
dataset_ids = [
    'business-establishments-with-address-and-industry-classification'
]

datasets = {}
for dataset_id in dataset_ids:
    datasets[dataset_id] = download_dataset(dataset_id)
    if datasets[dataset_id] is not None:
        print(f"{dataset_id} downloaded successfully.")

business-establishments-with-address-and-industry-classification downloaded successfully.


In [ ]:
datasets['business-establishments-with-address-and-industry-classification'].head()

,census_year,block_id,property_id,base_property_id,clue_small_area,trading_name,business_address,industry_anzsic4_code,industry_anzsic4_description,longitude,latitude
0,2003,105,100172,100172,Melbourne (CBD),Wilson Parking Australia,24-46 A'Beckett Street MELBOURNE 3000,9533,Parking Services,144.962053,-37.808573
1,2003,105,103301,103301,Melbourne (CBD),Melbourne International Backpackers,442-450 Elizabeth Street MELBOURNE 3000,4400,Accommodation,144.960868,-37.808309
2,2003,105,103302,103302,Melbourne (CBD),Vacant,422-440 Elizabeth Street MELBOURNE 3000,0,Vacant Space,144.961017,-37.808630
3,2003,105,103302,103302,Melbourne (CBD),The Garden Cafe,"Shop 3, Ground , 422-440 Elizabeth Street MELB...",4511,Cafes and Restaurants,144.961017,-37.808630
4,2003,105,103302,103302,Melbourne (CBD),Telephony Australia,"Shop 5, Ground , 422-440 Elizabeth Street MELB...",5809,Other Telecommunications Services,144.961017,-37.808630


In [ ]:
shape_of_dataset = datasets['business-establishments-with-address-and-industry-classification'].shape
print(shape_of_dataset)

(374210, 11)


In [ ]:
# Save
base_path = '/content/drive/My Drive/sit323'

for dataset_id, df in datasets.items():
    if df is not None:
        filename = f"{base_path}{dataset_id}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved {filename} to Google Drive.")

Saved /content/drive/My Drive/sit323business-establishments-with-address-and-industry-classification.csv to Google Drive.
